# Foundation of Computer Science
## Progetto d'esame


In [48]:
'''
 Librerie utilizzate all'interno del progetto
'''

import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.parser import parse
import os

## Definizione delle funzioni

In [3]:
'''
Conversione dell'attributo "Size" in un numero
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Creazione di un dizionario avente come chiavi i suffissi e come valore il moltiplicatore associato
    - Definizione della funzione per la conversione
'''

string_to_num_size = re.compile('(?P<val>\d+\.{0,1}\d*)(?P<unit>[\w{0,1}])')

unit_to_val = {'M' : 1000000,
               'k' : 1000,
               #'+' : 1000, #non è più necessario
               'Varies with device': np.nan}

def size_to_num(value):
    pippo = string_to_num_size.search(value.replace(",", ".")) #replace serve per sistemare formattazione errata di alcuni valori
    if(pippo != None):
        num = float(pippo.group('val'))
        return int(num)*unit_to_val[pippo.group('unit')]
    else:   
        return unit_to_val[value]
    


In [4]:
'''
Conversione dell'attributo "Installs" in un numero
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Definizione della funzione per la conversione
'''

string_to_num_installs = re.compile('(?P<val>\d+)(?P<plus>\+{0,1})')

def installs_to_num(value):
    pluto = string_to_num_installs.search(value.replace(",","")) #replace serve per sistemare formattazione errata di alcuni valori
    if(pluto != None):
        return int(pluto.group('val'))
    else:
        return np.nan

In [5]:
'''
Conversione degli attributi "Current Ver" e "Android Ver" in dotted number
    
    - Definizione dell'espressione regolare utilizzata per fare il parsing del valore
    - Definizione della funzione per la conversione
'''


'''
    Current Ver: 
        - Troppe varianti per creare un'espressione regolare che matchi sempre
        - Cambio approccio, pattern che matcha dotted notation e la estrae dal campo
        - Se la trova la restituisce, tutto ciò che non è scritto come dotted_notation lo converto a 1.0
        
    Android Ver: Stesso approccio di Current Ver, trovo pattern dottet notation nel campo e lo restituisco,
                 Se non c'è restituisco 1.0, se ho più versioni restituisco l'ultima,
'''

dotted_notation = re.compile('(?P<num>\d+(\.\d+)*)')

def clean_current_ver(value):
    if(pd.isnull(value)):
        return '1.0'
    else:
        donald = dotted_notation.search(value)
        if(donald != None):
            return donald.group('num')
        else:
            return '1.0'
        
def clean_android_ver(value):
    if(value == 'Varies with device')
        daisy = dotted_notation.findall(value)
        if(daisy != None):
            return daisy[-1][0]
        else:
            return '1.0'

In [6]:
'''
Rimozione dei duplicati e funzioni di supporto

    La funzione prende in ingresso il dataframe da controllare e ne restituisce uno nuovo senza duplicati
    Per ogni nome di app estra un sottoinsieme del dataframe di partenza.
    Se il numero di righe è pari a 1 la aggiunge al nuovo dataframe, se è diverso viene effettuato
    un primo controllo utilizzando un metodo SQL-like tramite GroupBy.
    Se due righe hanno lo stesso valore per Current Ver, Android Ver e Last Updated mantendo solamente quella
    con il numero di reviews più alto (tramite la funzione lambda).
    Controllo il numero di righe che ha il sotto-dataframe che si è venuto a creare, se è pari a 1 ho eliminato
    i duplicati e aggiungo la riga al nuovo dataframe altrimenti effettuo un ulteriore controllo tramite la
    funzione check_update_version.
    Questa funzione ha il compito di controllare se le righe del sotto-dataframe rimanente si riferiscano ad
    app diverse, ma con stesso nome, oppure a versioni differenti della stessa app. Alla funzione viene passato
    il sotto-dataframe ordinato in maniera crescente di "Last Updated".
    Per controllare se siano più versioni o app diverse sono state create delle funzioni di supporto.
    is_same_genre serve per controllare che le app appartengano allo stesso genere.
    is_previous_update serve per controllare che l'attributo "last updated" dell'ultima riga sia il più recente
    is_next_version serve per controllare che la "Current Ver" dell'ultima riga sia più aggiornata rispetto alle altre righe
    has_more_reviews serve per controllare che l'ultima riga abbia un numero di reviews più alto delle altre
    Se queste condizioni sono soddisfatte allora mantendo solamente la riga corrispondente alla versione più
    aggiornata, altrimenti le app sono effettivamente diverse e mantengo tutte le righe nel nuovo dataframe.
    
'''

def remove_duplicate_rows(old_df):
    new_df = pd.DataFrame(columns=old_df.columns)
    app_name = old_df['App'].unique()
    for app in app_name:    
        tmp_df = old_df[old_df['App'] == app] 
        if(tmp_df.shape[0] != 1):            
            tmp_df = tmp_df.groupby(['Current Ver', 'Android Ver', 'Last Updated'], group_keys=False).apply(lambda x: x.loc[x['Reviews'].idxmax()]).reset_index(drop=True)           
            if(tmp_df.shape[0] != 1): 
                new_df.append(check_update_version(tmp_df.sort_values(by='Last Updated'))) 
            else:
                new_df = new_df.append(tmp_df)
        else:
            new_df = new_df.append(tmp_df)
    return new_df.reset_index(drop=True)

def check_update_version(to_check):
    if(len(to_check['Type'].unique()) == 1 and
      is_same_genre(to_check['Genres'].tolist()) and
      is_previous_update(to_check['Last Updated'].tolist()) and
      is_next_version(to_check['Current Ver'].tolist()) and
      has_more_reviews(to_check['Reviews'].tolist())): #allora è la versione aggiornata
        return to_check.iloc[-1:,]
    else:
        return to_check

def is_same_genre(genres):
    if(min(genres, key=len) in max(genres, key=len)): 
        return True
    else:
        return False
        
def is_previous_update(dates):
    if(max(dates) == dates[-1]): 
        return True
    else:
        return False
    
def is_next_version(versions): 
    non_digit = re.compile(r'[^\d]')
    new_versions = [re.sub(non_digit, '', str(x)) for x in versions]
    if(max(new_versions) == new_versions[-1]):
        return True
    else:
        return False
    
def has_more_reviews(reviews): 
    if(max(reviews) == reviews[-1]):
        return True
    else:
        return False

In [63]:
'''
 Caricamento del percorso dei dataset
'''

gps_path = os.getcwd() + "\\googleplaystore.csv"
gps_ur_path = os.getcwd() + "\\googleplaystore_user_reviews.csv"

In [64]:
gps = pd.read_csv(gps_path, low_memory = False)
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [65]:
'''
Operazioni preliminari sul Dataset googleplaystore

    - Eliminazione della riga numero 10472 in quanto i campi sono formattati in modo errato
    - Eliminazione delle righe in cui 'Android Ver' è un missing value
    
    - Eliminazione delle righe in cui i campi "Current Ver" e "Android Ver" non sono presenti
      o sono uguali a "Varies with device"
      
    - Eliminazione dei duplicati attraverso la funzione drop_duplicated() di pandas, elimina 
      solamente le tuple con tutti gli attributi uguali
    - Conversione dell'attributo "Reviews" a numeric
    - Conversione dell'attributo "Last Updated" a datetime
    - Conversione dell'attributo "Rating" a numeric

      
'''

gps = gps.drop(gps.index[10472])
gps = gps.dropna(subset=['Android Ver'])
gps = gps.drop_duplicates()
gps = gps[gps['Android Ver'].notnull()]
#gps = gps[gps['Android Ver'] != 'Varies with device']
#gps = gps[gps['Current Ver'] != 'Varies with device']


gps['Reviews'] = pd.to_numeric(gps['Reviews'], downcast = 'float')
gps['Last Updated'] = pd.to_datetime(gps['Last Updated'])
gps['Rating'] = pd.to_numeric(gps['Rating'], downcast='float')

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19M,"10,000+",Free,0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25M,"50,000,000+",Free,0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167.0,5.6M,"50,000+",Free,0,Everyone,Art & Design,2017-03-26,1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178.0,19M,"50,000+",Free,0,Everyone,Art & Design,2018-04-26,1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815.0,29M,"1,000,000+",Free,0,Everyone,Art & Design,2018-06-14,6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791.0,33M,"1,000,000+",Free,0,Everyone,Art & Design,2017-09-20,2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121.0,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,2018-07-03,2.8,4.0.3 and up


In [71]:
'''
    - Applicazione delle funzioni definite precedentemente per i punti da 1 a 5
    - Eliminazione delle colonne non più necessarie
    - Rename delle colonne sistemate
    - Cast degli attributo al tipo corretto
    
'''

#gps['IntSize'] = gps['Size'].apply(size_to_num)
#gps['IntInstalls'] = gps['Installs'].apply(installs_to_num)
gps['Android Ver ok'] = gps['Android Ver'].apply(clean_android_ver)
#gps['Current Ver ok'] = gps['Current Ver'].apply(clean_current_ver)

#cleaned_gps = remove_duplicate_rows(gps)

#to_drop = ['Size', 'Installs', 'Android Ver', 'Current Ver']
#cleaned_gps.drop(to_drop, axis = 1, inplace=True)

#new_columns = ['App', 'Category','Rating','Reviews','Type','Price','Content Rating','Genres','Last Updated',
#               'Size', 'Installs', 'Android Ver', 'Current Ver']
#cleaned_gps.columns = new_columns

#cleaned_gps['Installs'] = pd.to_numeric(cleaned_gps['Installs'], downcast='integer')
#cleaned_gps['Size'] = pd.to_numeric(cleaned_gps['Size'], downcast='integer')
#cleaned_gps['Reviews'] = pd.to_numeric(cleaned_gps['Reviews'], downcast='integer')
#cleaned_gps['Android Ver'] = cleaned_gps['Android Ver'].apply(str)
#cleaned_gps['Current Ver'] = cleaned_gps['Current Ver'].apply(str)

IndexError: list index out of range

In [11]:
cleaned_gps.head()

,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size,Installs,Android Ver,Current Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,Free,0,Everyone,Art & Design,2018-01-07,19000000.0,10000,4.0.3,1.0.0
1,Coloring book moana,FAMILY,3.9,974,Free,0,Everyone,Art & Design;Pretend Play,2018-01-15,14000000.0,500000,4.0.3,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,Free,0,Everyone,Art & Design,2018-08-01,8000000.0,5000000,4.0.3,1.2.4
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,Free,0,Everyone,Art & Design;Creativity,2018-06-20,2000000.0,100000,4.4,1.1
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,Free,0,Everyone,Art & Design,2017-03-26,5000000.0,50000,2.3,1.0


In [12]:
'''
    Creazione del DataFrame Category:
        - Categoria
        - Numero di app all'interno della categoria
        - Valutazione media delle app all'interno della categoria
'''

#Dichiarazione delle liste per la creazione del DataFrame Category
category_name = cleaned_gps['Category'].unique()
category_num = []
category_rating = []

#Estrazione dei dati per popolare il DataFrame
for cat in category_name:
    category_num.append(len(cleaned_gps[cleaned_gps['Category'] == cat]))
    category_rating.append(round(cleaned_gps[cleaned_gps['Category'] == cat]['Rating'].mean(),2))

#Creazione del DataFrame Category
category_df = pd.DataFrame({'Category': category_name, 'Numero': category_num, 'Avg Rating': category_rating}).sort_values('Avg Rating', ascending=False)

In [13]:
category_df.head()

,Category,Numero,Avg Rating
11,EVENTS,55,4.49
9,EDUCATION,80,4.37
0,ART_AND_DESIGN,58,4.36
4,BOOKS_AND_REFERENCE,196,4.33
26,PERSONALIZATION,346,4.32


In [14]:
'''
    Creazione dei due dataframe, uno contenente i diversi generi di app presenti all'interno del dataframe originario
    e uno contenente le diverse app con il corrispondente o i corrispondenti generi.
    
    DataFrame Genres: per ogni genere è riportato la media delle valutazioni corrispondenti ordinate in ordine decrescente
    
    DataFrame App-Genres: per ogni app è riportato il nome, il genere.

'''

#Dichiarazione delle liste per la creazione del DataFrame Generi
genre_list = []
avg_rating_gen = []

#Dichiarazione delle liste per la creazione del DataFrame App-Genere
app_list = []
genre_app_list = []
rating_list = []

#Dichiarazione dei pattern per l'estrazione dei generi
semicolumn = re.compile('[;]')

#Estrazione dal DataFrame originario dei dati per popolare i due nuovi DataFrame
for index, row in cleaned_gps.iterrows():
    double_genre = semicolumn.search(row['Genres'])
    if(double_genre != None):
        list_gen = row['Genres'].split(';')
        for genre in list_gen:
            if(genre not in genre_list):
                genre_list.append(genre)
            app_list.append(row['App'])
            genre_app_list.append(genre)
            rating_list.append(row['Rating'])            
    else:
        if(row['Genres'] not in genre_list):
            genre_list.append(row['Genres'])
        app_list.append(row['App'])
        genre_app_list.append(row['Genres'])
        rating_list.append(row['Rating'])
        
#Creazione del DataFrame App-Genre
app_genre_df = pd.DataFrame({'App': app_list, 'Genre': genre_app_list, 'Rating': rating_list}).sort_values('Rating', ascending=False)

#Utilizzo del DataFrame appena creato per calcolare la media delle valutazioni per genere
for genre in app_genre_df['Genre'].unique():
    avg_rating_gen.append(round(app_genre_df[app_genre_df['Genre'] == genre]['Rating'].mean(),2))
    
#Creazione del DataFrame Genres
genres_df = pd.DataFrame({'Genre': genre_list, 'Avg Rating': avg_rating_gen}).sort_values('Avg Rating', ascending = False)

In [15]:
genres_df.head()

,Genre,Avg Rating
15,Events,4.49
20,Libraries & Demo,4.37
27,Puzzle,4.36
40,Social,4.35
45,Personalization,4.34


In [16]:
app_genre_df.head()

,App,Genre,Rating
8880,Fr. Mike Schmitz Audio Teachings,Education,5.0
5684,Trovami se ci riesci,Arcade,5.0
6171,CT Checkout,Finance,5.0
6279,Morse Player,Education,5.0
6282,30WPM Amateur ham radio Koch CW Morse code tra...,Education,5.0


In [17]:
'''
    Per ogni genere presente all'interno del DataFrame creare una colonna con valori booleani.
    Il valore presente all'interno di queste nuove colonne sarà uguale a True solamente nel caso in cui
    corrisponda al genere dell'app di riferimento
'''

#creazione delle nuove colonne del datafra
for gen in genres_df['Genre']:
    cleaned_gps[gen] = False
    
for index,row in cleaned_gps.iterrows():
    row_gen = row['Genres']
    if(semicolumn.search(row_gen) != None):
        for g in row_gen.split(';'):
            cleaned_gps.loc[index, g] = True
    else:
        cleaned_gps.loc[index, row_gen] = True

In [18]:
cleaned_gps.head()

,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size,...,Parenting,Strategy,Education,Card,Adventure,Role Playing,House & Home,Comics,Board,Music
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,Free,0,Everyone,Art & Design,2018-01-07,19000000.0,...,False,False,False,False,False,False,False,False,False,False
1,Coloring book moana,FAMILY,3.9,974,Free,0,Everyone,Art & Design;Pretend Play,2018-01-15,14000000.0,...,False,False,False,False,False,False,False,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,Free,0,Everyone,Art & Design,2018-08-01,8000000.0,...,False,False,False,False,False,False,False,False,False,False
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,Free,0,Everyone,Art & Design;Creativity,2018-06-20,2000000.0,...,False,False,False,False,False,False,False,False,False,False
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,Free,0,Everyone,Art & Design,2017-03-26,5000000.0,...,False,False,False,False,False,False,False,False,False,False


In [19]:
'''
    Aggiunta al DataFrame iniziale di due colonne:
        - Income: guadagno dell'app (price*installs)
        - Rapporto tra numero di installazioni e reviews
    
    Conversione dell'attributo Price da String a Float

'''

#Dichiarazione pattern per l'estrazione del prezzo
price = re.compile('(?P<num>\d+\.{0,1}\d*)')

#Definizione funzione per il calcolo dell'income, da applicare al DataFrame per ogni riga
def compute_income(row):
    pr = price.search(str(row['Price']))
    if(pr != None):
        income = float(pr.group('num'))*row['Installs']
        return income
    else:
        return '0'
  
#Definizione funzione per la conversione dell'attributo Price da stringa a float
def price_to_float(row):
    pr = price.search(str(row['Price']))
    if(pr != None):
        return float(pr.group('num'))
    else:
        return 0.0
    
#Definizione funzione per il calcolo del rapporto Installazioni/Reviews
def install_reviews_ratio(row):
    if(row['Reviews'] == 0):
        return 0
    else:
        ratio = round((row['Installs']/row['Reviews']),2)
        return ratio

cleaned_gps['Income'] = cleaned_gps.apply(compute_income, axis = 1)
cleaned_gps['Income'] = pd.to_numeric(cleaned_gps['Income'], downcast='float')
cleaned_gps['Price'] = cleaned_gps.apply(price_to_float, axis = 1)
cleaned_gps['InRev Ratio'] = cleaned_gps.apply(install_reviews_ratio, axis = 1)
cleaned_gps['InRev Ratio'] = pd.to_numeric(cleaned_gps['InRev Ratio'], downcast='float')

In [20]:
cleaned_gps.head()

,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size,...,Education,Card,Adventure,Role Playing,House & Home,Comics,Board,Music,Income,InRev Ratio
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,Free,0.0,Everyone,Art & Design,2018-01-07,19000000.0,...,False,False,False,False,False,False,False,False,0.0,62.889999
1,Coloring book moana,FAMILY,3.9,974,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,14000000.0,...,False,False,False,False,False,False,False,False,0.0,513.349976
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,Free,0.0,Everyone,Art & Design,2018-08-01,8000000.0,...,False,False,False,False,False,False,False,False,0.0,57.139999
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,2000000.0,...,False,False,False,False,False,False,False,False,0.0,103.410004
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,Free,0.0,Everyone,Art & Design,2017-03-26,5000000.0,...,False,False,False,False,False,False,False,False,0.0,299.399994


In [21]:
'''
    Prezzo medio per Content Rating delle app non gratuite
'''
avg_content_price = []

for cr in cleaned_gps['Content Rating'].unique():
    avg_content_price.append(round(cleaned_gps[(cleaned_gps['Content Rating'] == cr) & (cleaned_gps['Type'] != 'Free')]['Price'].mean(),2))

cr_df = pd.DataFrame({'Content Rating': cleaned_gps['Content Rating'].unique(), 'Avg Price': avg_content_price}).sort_values('Avg Price', ascending=False)

In [22]:
cr_df

,Content Rating,Avg Price
0,Everyone,16.01
1,Teen,12.77
3,Mature 17+,5.71
2,Everyone 10+,4.38
4,Adults only 18+,NaN
5,Unrated,NaN


In [23]:
'''
    Clustering delle app secondo la versione di Android (prime due cifre)
    Calcolo della data media e dell'ultima data degli aggiornamenti
'''

#estrazione di tutte le versioni di android (2 cifre) presenti nel DataFrame e creazione del dizionario
# per l'assegnazione del cluster ad ogni riga
android_ver = []
avg_date = []
last_update = []
n_app = []
for i in cleaned_gps['Android Ver'].unique(): 
    if(i[:3] not in android_ver):
        android_ver.append(i[:3])
    
cluster_num = [i for i in np.arange(1,len(android_ver)+1,1)]
android_dict = dict(zip(android_ver,cluster_num))
    
#assegnazione delle righe ai rispettivi cluster
cleaned_gps['Cluster N'] = cleaned_gps['Android Ver'].apply(lambda x: android_dict[x[:3]])

#calcolo della data media per cluster e dell'ultima data degli aggiornamenti
for n in cleaned_gps['Cluster N'].unique():
    start = cleaned_gps[cleaned_gps['Cluster N'] == n]['Last Updated'].min()
    end = cleaned_gps[cleaned_gps['Cluster N'] == n]['Last Updated'].max()
    avg_date.append((start + (end - start)/2).strftime('%Y-%m-%d'))
    last_update.append(end)
    n_app.append(len(cleaned_gps[cleaned_gps['Cluster N'] == n]))
    
cluster_df = pd.DataFrame({'Android Ver': list(android_dict.keys()), 'Cluster N': list(android_dict.values()),
                           'N App': n_app, 'Mean Date': avg_date, 'Last Update': last_update})

In [36]:
cluster_df


,Android Ver,Cluster N,N App,Mean Date,Last Update
0,4.0,1,2637,2016-04-23,2018-08-08
1,4.4,2,813,2016-09-28,2018-08-07
2,2.3,3,876,2015-05-17,2018-08-04
3,4.2,4,364,2016-05-06,2018-08-08
4,3.0,5,229,2015-05-05,2018-08-02
5,4.1,6,2173,2016-06-14,2018-08-08
6,2.2,7,238,2015-04-26,2018-08-03
7,6.0,8,52,2017-06-19,2018-08-04
8,5.0,9,495,2016-11-10,2018-08-08
9,1.6,10,116,2014-11-02,2018-08-06


In [25]:
'''
Caricamento del dataset contenente le reviews del playstore
'''
gps_ur = pd.read_csv('googleplaystore_user_reviews.csv', low_memory = False)
gps_ur.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [26]:
'''
Per ogni app calcolare:
    - Sentiment polarity massima e minima
    - lunghezza media delle reviews
    - review più lunga
    
    - Conversione della colonna "Sentiment_Polarity" da notazione scientifica a decimale con 5 cifre significative
    - Definizione della funzione per estrarre la lunghezza delle reviews
    - Applicazione della funzione al dataset
    - Creazione del DataFrame con:
        - App
        - Lunghezza media delle review
        - Lunghezza della review più lunga
        - Review più lunga
        - Sentiment Polarity massima
        - Sentiment Polarity minima
    
'''

def get_length_review(text):
    if(pd.isnull(text)):
        return 0
    else:
        return len(text.split())

gps_ur['Sentiment_Polarity'] = gps_ur['Sentiment_Polarity'].apply(lambda x: "{:.4f}".format(x))    
gps_ur['Review Len'] = gps_ur['Translated_Review'].apply(get_length_review)
gps_ur['Review Len'] = pd.to_numeric(gps_ur['Review Len'], downcast = 'integer')

avg_text = []
longest_rew = []
longest_text = []
max_polarity = []
min_polarity = []

for app in gps_ur['App'].unique():
    avg_text.append(round(gps_ur[gps_ur['App']==app]['Review Len'].mean(),2))
    longest_rew.append(gps_ur[gps_ur['App']==app]['Review Len'].max())
    longest_text.append(gps_ur[(gps_ur['App']==app)& (gps_ur['Review Len'].max())]['Translated_Review'].iloc[0,])
    max_polarity.append(gps_ur[(gps_ur['Sentiment'] == 'Positive') & (gps_ur['App'] == app)]['Sentiment_Polarity'].max())
    min_polarity.append(gps_ur[(gps_ur['Sentiment'] == 'Negative') & (gps_ur['App'] == app)]['Sentiment_Polarity'].max())
app_ur_df = pd.DataFrame({'App': gps_ur['App'].unique(), 'Avg Review Len': avg_text, 
                          'Max Len': longest_rew , 'Max Text': longest_text,
                          'Max Polarity': max_polarity, 'Min Polarity': min_polarity}).sort_values('Avg Review Len', ascending = False)

In [27]:
app_ur_df.head()

,App,Avg Review Len,Max Len,Max Text,Max Polarity,Min Polarity
513,Crunchyroll - Everything Anime,44.72,64,Great app. Works like charm me. Main complaint...,0.6500,-0.4250
1001,Hacker's Keyboard,44.12,105,"Some damn fine work. I'm novelist, far best ke...",0.5567,-0.3167
1019,Helix Jump,40.67,102,"Actually really enjoy playing game, however I ...",0.5236,-0.2385
223,BeWild Free Dating & Chat App,40.15,82,Wow amazing dating app. The registration proce...,0.5300,-0.5208
447,Clash Royale,38.63,70,The game favors big spenders rigged deck rotat...,0.5000,-0.3833


array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)